In [1]:
#imports
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.simplefilter("ignore")

In [2]:
#code was inpsired by AssemblyAI - patric

In [3]:
#import sklean with version cheack
import sklearn
sklearn.__version__

'1.2.0'

In [4]:
#getting the data
url = 'https://raw.githubusercontent.com/basil-b2s/Language-Detector/master/language_detection.csv'
data = pd.read_csv(url)

In [5]:
#divding the data
X = data["Text"]
y = data["Language"]

In [6]:
#makes the y as labels 
# Arabic = 0
# Danish = 1 
# etc..
from sklearn.preprocessing import LabelEncoder
lebel = LabelEncoder()
y = lebel.fit_transform(y)

In [7]:
lebel.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [8]:
#removing with regex remove speical marks
data_list = []
for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [9]:
#train & test split (25/75 was the best result)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [10]:
# creating bag of words using countvectorizer

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(X_train)

x_train = cv.transform(X_train).toarray()
x_test  = cv.transform(X_test).toarray()

In [11]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [12]:
y_pred = model.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

In [14]:
print("Accuracy is :",accuracy)

Accuracy is : 0.9740812379110252


In [15]:
#makeing 2 modles to 1
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vectorizer', cv), ('multinomialNB', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [16]:
y_pred2 = pipe.predict(X_test)
ac2 = accuracy_score(y_test, y_pred2)
print("Accuracy is :",ac2)

Accuracy is : 0.9740812379110252


In [17]:
with open('trained_pipeline-0.5.0.pkl','wb') as f:
    pickle.dump(pipe, f)

In [18]:
!zip -r ./trained_pipeline-0.5.0.pkl.zip ./trained_pipeline-0.5.0.pkl

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
text = "face"
y = pipe.predict([text])
lebel.classes_[y[0]], y

('English', array([3]))